In [1]:
import requests
import urllib3
import json
from lxml import html, etree
import requests.packages.urllib3.exceptions
from urllib3.exceptions import InsecureRequestWarning
import time
import re
import datetime

In [2]:
def getRequest(url):
    headers = {
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept-Language': 'en-GB,en;q=0.9,en-US;q=0.8,ml;q=0.7',
        'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.140 Safari/537.36'
    }
    requests.packages.urllib3.disable_warnings(InsecureRequestWarning)
    response = requests.get(url, verify=False, headers=headers)
    return response.text

In [13]:
def getArticleURLs(countryURL, numOfPages):
    urls = []
    
    #scrape the front page
    parser = html.fromstring(getRequest(countryURL))
    items = parser.xpath("//div[@class='fc-item__content ']")
    for item in items:
        for elt in item.xpath(".//h3[@class='fc-item__title']"):
            urls.append(elt[0].attrib['href'])
            
    #scrape the rest of the pages
    for i in range(1, numOfPages):
        #https://www.theguardian.com/world/china?page=1000
        parser = html.fromstring(getRequest(countryURL + "?page=" + str(i)))

        #<div class="fc-item__content "> 
        items = parser.xpath("//div[@class='fc-item__content ']")

        for item in items:
          for elt in item.xpath(".//h3[@class='fc-item__title']"):
            urls.append(elt[0].attrib['href'])

    return urls

In [8]:
def month_converter(month):
    months = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']
    return months.index(month) + 1

def dateToList(dateList):
    dateList[0] = int(dateList[0])
    dateList[1] = month_converter(dateList[1])
    dateList[2] = int(dateList[2])
    return dateList

def isYear(string):
    if re.search("(?=.*[1][9][0-9][0-9]|[2][0-9][0-9][0-9])", string) is not None:
        return True
    else:
        return False
    
#ignoring non-articles with little or no text
def vetUrl(url):
    sure = ["video","gallery","gallery","audio","picture","us-embassy-cables-documents"]
    unsure = ["blog","from-the-archive-blog","radical-conservation","lostinshowbiz","the-running-blog","australia-books-blog"]
    for s in sure:
        if s in url:
            return False
    for u in unsure:
        if u in url:
            printStr = printStr + "------------------------------\n" + url
            return False
    return True

In [16]:
def scrapeCategory(urlList):
    dfRows = []
    printStr = ""
    
    for u in urlList:
        url = ""
        #vet url
        if vetUrl(u):
            url = u
        else:
            continue   
                
        newsParser = html.fromstring(getRequest(url))
        #print(url)
        #print title test
        #print(newsParser.xpath(".//h1")[0].text.strip())

        #interactive articles that we don't need
        if "ng-interactive" in url or "None" in url:
            continue
        #specially formatted Guardian article
        elif newsParser.xpath(".//h1")[0].text == None:
            for elt in newsParser.xpath(".//h1"):
                #print(url)
                data["title"] = elt[0].text.strip()
        #normal Guardian article
        else:
            data["title"] = newsParser.xpath(".//h1")[0].text.strip()

        #when we get kicked out the h1 (title) changes
        if '429 Too Many Requests' in data["title"]:
            print("Too many requests!")
            print("Data gathered: ", len(dfRows))
            break

        #url is simple
        data["url"] = url

        #date
        #get all elements of the url and loop through them
        elements = u.rstrip().split('/')
        for i in range(0, len(elements)):
            #if a i == year, it means a date follows [i:i+3]
            if elements[i].isnumeric() and isYear(elements[i]):
                try:
                    #date always starts from i
                    data["date"] = dateToList(elements[i:i+3])
                #in case we give the date function a bad string
                except IndexError:
                    print(url)
                #in case we find a new sub-category 
                except ValueError:
                    print(dateList[0])
                    continue

        #get article text - for Guardian - always stored in <p>
        #collect all paragraphs
        paragraphList = []
        for item in newsParser:
            for elt in item.xpath(".//p"):
                paragraphList.append(elt.text)

        #first one is always the subtitle
        data["subtitle"] = paragraphList.pop(0)

        #collect raw article text
        data["text"] = ''
        for paragraph in paragraphList:
            #print(paragraph)
            #if it's None for some reason: continue
            if isinstance(paragraph, str) is False:
                continue
            #Don't need this: we hve published date
            elif "Last modified on " in paragraph:
                continue
            #some times <p> are something weird like above;
            try:
                data["text"] = data["text"] + paragraph.strip()
            #so if it is: split it out and continue
            except TypeError:
                print("Type Error!")
                print(paragraph)
                continue

        #write data entry to list
        dfRows.append(data)
        #continue after a delay, so we don't go over request limit; can't be lower than this
        time.sleep(900/1000)

    #when done
    print(printStr)
    print(str(len(dfRows)) + " entries!")
    print("Scraping finished!")

In [44]:
#date = "Last modified on Wed 7 Apr 2021 16.37 BST"
#print(dateToList(date))

#newsParser = html.fromstring(getRequest("https://www.theguardian.com/world/ng-interactive/2020/dec/14/coronavirus-2020-timeline-covid-19"))
#newsParser = html.fromstring(getRequest("https://www.theguardian.com/world/2020/oct/26/china-new-coronavirus-outbreak-detected-in-xinjiang-city-of-kashgar"))
#print(newsParser.xpath(".//h1"))
#for item in newsParser:
#      for elt in item.xpath(".//p"):
#        print(elt.text)

#print(len(dfRows))

In [6]:
import pandas as pd
import numpy as np

#input rows of data
#output a pandas dataframe
def saveToDf(dfRows):
    # create a new dataframe and parser
    df = pd.DataFrame()
    df = df.append(dfRows, ignore_index=True)
    return df

In [14]:
countryList = [#('China', 'https://www.theguardian.com/world/china', 1101),
                ('USA', 'https://www.theguardian.com/us-news', 1900),
                ('Germany', 'https://www.theguardian.com/world/germany', 622),
                ('Russia', 'https://www.theguardian.com/world/russia', 970),
                ('Japan', 'https://www.theguardian.com/world/japan', 370),
               ('North Korea', 'https://www.theguardian.com/world/north-korea', 244),
               ('South Korea', 'https://www.theguardian.com/world/south-korea', 143),
               ('Australia', 'https://www.theguardian.com/australia-news', 3566),
               ('India', 'https://www.theguardian.com/world/india', 544)]
dfs = []
for country in countryList:
    urls = getArticleURLs(country[1], country[2])
    dataRows = scrapeCategory(urls)
    dfs.append(saveToDf(dfRows))

KeyboardInterrupt: 

In [48]:
#print(len(df['tiltle']==None))
#print(len(df['url']==None))
#print(len(df['subtitle']==None))
#print(len(df['text']==None))
#print(df[df['url']=='https://www.theguardian.com/football/2019/dec/13/arsenal-distance-themselves-from-mesut-ozil-comments-china-uighur-people'])
#print(df[df['title']=='429 Too Many Requests'])
#temp = "https://www.theguardian.com/books/australia-books-blog/2019/apr/29/les-murray-remembered-by-john-kinsella-a-brilliant-battler-poet".split('/')
#temp = "https://www.theguardian.com/artanddesign/gallery/2015/feb/13/red-army-primary-school-gallery".split('/')#[5:8]
#print(temp)
dfUsa = dfs[0]
dfUsa.head(10)

,title,url,date,subtitle,text
0,Hong Kong activists plead guilty but say ‘hist...,https://www.theguardian.com/world/2021/apr/07/...,"[2021, 4, 7]",Lee Cheuk-yan says he and fellow accused Jimmy...,The prominentThe two were facing charges along...
1,Hong Kong activists plead guilty but say ‘hist...,https://www.theguardian.com/world/2021/apr/07/...,"[2021, 4, 7]",Lee Cheuk-yan says he and fellow accused Jimmy...,The prominentThe two were facing charges along...
2,Hong Kong activists plead guilty but say ‘hist...,https://www.theguardian.com/world/2021/apr/07/...,"[2021, 4, 7]",Lee Cheuk-yan says he and fellow accused Jimmy...,The prominentThe two were facing charges along...
3,Hong Kong activists plead guilty but say ‘hist...,https://www.theguardian.com/world/2021/apr/07/...,"[2021, 4, 7]",Lee Cheuk-yan says he and fellow accused Jimmy...,The prominentThe two were facing charges along...
4,Hong Kong activists plead guilty but say ‘hist...,https://www.theguardian.com/world/2021/apr/07/...,"[2021, 4, 7]",Lee Cheuk-yan says he and fellow accused Jimmy...,The prominentThe two were facing charges along...
5,Hong Kong activists plead guilty but say ‘hist...,https://www.theguardian.com/world/2021/apr/07/...,"[2021, 4, 7]",Lee Cheuk-yan says he and fellow accused Jimmy...,The prominentThe two were facing charges along...
6,Hong Kong activists plead guilty but say ‘hist...,https://www.theguardian.com/world/2021/apr/07/...,"[2021, 4, 7]",Lee Cheuk-yan says he and fellow accused Jimmy...,The prominentThe two were facing charges along...
7,Hong Kong activists plead guilty but say ‘hist...,https://www.theguardian.com/world/2021/apr/07/...,"[2021, 4, 7]",Lee Cheuk-yan says he and fellow accused Jimmy...,The prominentThe two were facing charges along...
8,Hong Kong activists plead guilty but say ‘hist...,https://www.theguardian.com/world/2021/apr/07/...,"[2021, 4, 7]",Lee Cheuk-yan says he and fellow accused Jimmy...,The prominentThe two were facing charges along...
9,Hong Kong activists plead guilty but say ‘hist...,https://www.theguardian.com/world/2021/apr/07/...,"[2021, 4, 7]",Lee Cheuk-yan says he and fellow accused Jimmy...,The prominentThe two were facing charges along...
